In [2]:
import git
repo = git.Repo("./", search_parent_directories=True)

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [4]:
import pandas as pd

In [5]:
import matplotlib.pyplot as plt

# Data Preparation

Directory

In [6]:
homedir = repo.working_dir
datadir = f"{homedir}/data/"

## international/health

Let's take a look at the health data from the international folder.

In [6]:
import pandas as pd
df = pd.read_csv(datadir + 'international/health/hospital-beds-per-1000-people.csv')

We can pivot it using countries and years:

In [7]:
df=df.dropna(subset=['Code'])

In [8]:
X_beds=df.pivot(index="Code",columns="Year",values="Hospital beds (per 1,000 people) (per 1,000 people)")
X_beds.head()

Year,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
Code,,,,,,,,,,,,,,,,,,,,,
AFG,0.170627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.42,0.42,0.4,0.4,NaN,0.5,NaN,NaN
AGO,2.061462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALB,5.102676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.00,2.9692,2.90,NaN,2.8,NaN,2.43,2.6,NaN,NaN
AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.70,2.6000,2.60,NaN,2.5,NaN,NaN,NaN,NaN,NaN
ARE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.88,NaN,1.86,1.90,NaN,NaN,NaN,1.1,NaN,NaN


Notice that all the records are earlier than 2014, so for inference purpose we can only resort to the last non-NaN column for each country, and assume that it hasn't significantly changed since then.

In [9]:
ind = {x:X_beds.loc[x].last_valid_index() for x in X_beds.index}
last_data_point_beds_per_mille = {x:X_beds.loc[x, ind[x]] for x in X_beds.index}

Actually, we can build a helper function to do this:

In [10]:
def to_last_record(X):
    ind = {x:X.loc[x].last_valid_index() for x in X.index}
    return pd.Series([X.loc[x, ind[x]] for x in X.index], index=X.index)

In [11]:
df_phys = pd.read_csv(datadir + 'international/health/physicians-per-1000-people.csv')
df_phys.head()

,Entity,Code,Year,"Physicians (per 1,000 people) (per 1,000 people)"
0,Afghanistan,AFG,1960,0.035
1,Afghanistan,AFG,1965,0.063
2,Afghanistan,AFG,1970,0.065
3,Afghanistan,AFG,1981,0.077
4,Afghanistan,AFG,1986,0.183


In [12]:
X_phys = df_phys.pivot(index="Entity",columns="Year",values="Physicians (per 1,000 people) (per 1,000 people)")

In [13]:
df_pneu = pd.read_csv(datadir + 'international/health/pneumonia-death-rates-age-standardized.csv')
df_pneu.head()

,Entity,Code,Year,Deaths - Lower respiratory infections - Sex: Both - Age: Age-standardized (Rate) (Rate)
0,Afghanistan,AFG,1990,164.811829
1,Afghanistan,AFG,1991,151.460290
2,Afghanistan,AFG,1992,127.896225
3,Afghanistan,AFG,1993,124.725141
4,Afghanistan,AFG,1994,134.410918


In [14]:
X_pneu = df_pneu.pivot(index="Entity",columns="Year",values="Deaths - Lower respiratory infections - Sex: Both - Age: Age-standardized (Rate) (Rate)")

In [15]:
df_heart = pd.read_csv(datadir + 'international/health/share-deaths-heart-disease.csv')
df_heart.head()

,Entity,Code,Year,Deaths - Cardiovascular diseases - Sex: Both - Age: All Ages (Percent) (%)
0,Afghanistan,AFG,1990,23.707752
1,Afghanistan,AFG,1991,23.490307
2,Afghanistan,AFG,1992,23.146918
3,Afghanistan,AFG,1993,21.154207
4,Afghanistan,AFG,1994,19.756144


In [16]:
X_heart = df_heart.pivot(index="Entity",columns="Year",values="Deaths - Cardiovascular diseases - Sex: Both - Age: All Ages (Percent) (%)")

In [17]:
df_smoke = pd.read_csv(datadir + 'international/health/share-of-adults-who-smoke.csv')
df_smoke.head()

,Entity,Code,Year,"Smoking prevalence, total (ages 15+) (% of adults)"
0,Albania,ALB,2000,34.8
1,Albania,ALB,2005,32.7
2,Albania,ALB,2010,31.2
3,Albania,ALB,2011,30.7
4,Albania,ALB,2012,30.2


In [18]:
X_smoke = df_smoke.pivot(index="Entity",columns="Year",values="Smoking prevalence, total (ages 15+) (% of adults)")

We should also take a look at which years are polled:

In [19]:
from collections import Counter
for data in [X_beds, X_phys, X_pneu, X_heart, X_smoke]:
    print("")
    print(Counter([data.loc[x].last_valid_index() for x in data.index]))


Counter({2012: 62, 2011: 62, 2010: 28, 2009: 12, 2006: 9, 2005: 7, 1996: 4, 1990: 2, 2004: 2, 1970: 2, 2007: 2, 2008: 2, 1985: 1, 2014: 1, 1981: 1, 1997: 1, 2013: 1, 2001: 1})

Counter({2013: 59, 2014: 48, 2015: 43, 2010: 27, 2016: 21, 2012: 10, 2009: 7, 2011: 7, 2004: 7, 1999: 5, 2001: 5, 1995: 4, 1997: 2, 1981: 1, 2000: 1, 1980: 1, 2006: 1, 1998: 1, 2005: 1, 2003: 1, 1987: 1})

Counter({2017: 231})

Counter({2017: 231})

Counter({2016: 186, 2014: 1})


The last three looks good, but the first two data contain results from a long span of time. This means some data can be quite inaccurate if used for inference. We can try to extrapolate, but let's leave that to the future.

In [20]:
import pygal.maps

In [21]:
last_data_point_beds_per_mille

{'AFG': 0.5,
 'AGO': 0.8,
 'ALB': 2.6,
 'AND': 2.5,
 'ARE': 1.1,
 'ARG': 4.7,
 'ARM': 3.9,
 'ATG': 2.1,
 'AUS': 3.9,
 'AUT': 7.6,
 'AZE': 4.7,
 'BDI': 1.9,
 'BEL': 6.5,
 'BEN': 0.5,
 'BFA': 0.4,
 'BGD': 0.6,
 'BGR': 6.4,
 'BHR': 2.1,
 'BHS': 2.9,
 'BIH': 3.5,
 'BLR': 11.3,
 'BLZ': 1.1,
 'BMU': 6.3000001907000005,
 'BOL': 1.1,
 'BRA': 2.3,
 'BRB': 6.2,
 'BRN': 2.8,
 'BTN': 1.8,
 'BWA': 1.8,
 'CAF': 1.0,
 'CAN': 2.7,
 'CHE': 5.0,
 'CHL': 2.1,
 'CHN': 3.8,
 'CIV': 0.4,
 'CMR': 1.3,
 'COD': 0.8,
 'COG': 1.6,
 'COL': 1.5,
 'COM': 2.2,
 'CPV': 2.1,
 'CRI': 1.2,
 'CUB': 5.3,
 'CYM': 3.0,
 'CYP': 3.5,
 'CZE': 6.8,
 'DEU': 8.2,
 'DJI': 1.4,
 'DMA': 3.8,
 'DNK': 3.5,
 'DOM': 1.7,
 'DZA': 1.7,
 'ECU': 1.6,
 'EGY': 0.5,
 'ERI': 0.7,
 'ESP': 3.1,
 'EST': 5.3,
 'ETH': 6.3,
 'FIN': 5.5,
 'FJI': 2.1,
 'FRA': 6.4,
 'FSM': 3.2,
 'GAB': 6.3,
 'GBR': 2.9,
 'GEO': 2.6,
 'GHA': 0.9,
 'GIN': 0.3,
 'GMB': 1.1,
 'GNB': 1.0,
 'GNQ': 2.1,
 'GRC': 4.8,
 'GRD': 3.5,
 'GRL': 14.353400230407699,
 'GTM': 0.6,
 'GUY':

In [22]:
min(last_data_point_beds_per_mille.values())

0.1

In [23]:
import numpy as np

In [24]:
import pycountry
maps = {}
for keys in last_data_point_beds_per_mille.keys():
    tmp = pycountry.countries.get(alpha_3=keys)
    try:
        maps[tmp.alpha_2.lower()] = (last_data_point_beds_per_mille[keys])
    except:
        pass

In [25]:
worldmap_chart = pygal.maps.world.World()
worldmap_chart.title = 'Beds per mille'
worldmap_chart.add('Latest available', maps)
worldmap_chart.render_to_png('worldbeds.png')

ImportError: /home/tzuchen/PycharmProjects/SKTW/venv/lib64/python3.8/site-packages/_cffi_backend.cpython-38-x86_64-linux-gnu.so: file too short

### Correlations

In [ ]:
df_total = pd.DataFrame()
df_total['beds'] = to_last_record(X_beds)
df_total['phys'] = to_last_record(X_phys)
df_total['pneu'] = to_last_record(X_pneu)
df_total['heart'] = to_last_record(X_heart)
df_total['smoke'] = to_last_record(X_smoke)

In [ ]:
df_total.head()

Note that some entries are invalid. This is because the original dataset might not contain some countries at all.

Given the data we have for each country, let us plot the correlation matrix:

In [ ]:
import matplotlib.pyplot as plt

plt.matshow(df_total.corr(), vmin=-1, vmax=1, cmap="bwr")
plt.colorbar()
plt.xticks(range(df_total.shape[1]), df_total.columns)
plt.yticks(range(df_total.shape[1]), df_total.columns)
plt.show()

## Covid-19 cases

In [ ]:
df_covid=pd.read_csv(datadir+'international/covid/our_world_in_data/full_data.csv')
df_covid.head()

In [ ]:
Y_d=df_covid.pivot(index="location",columns="date",values="total_deaths")
Y_i=df_covid.pivot(index="location",columns="date",values="total_cases")

Let's see the top 100 countries with the most confirmed cases:

In [ ]:
Y_i.sort_values(by='2020-04-14').iloc[-100:-1]

In [ ]:
Y_i.sort_values(by='2020-04-14').iloc[-10:-2].T.plot()

# SEIR

## Fitting parameters

In [ ]:
df_pop = pd.read_csv(datadir + 'us/demographics/county_populations.csv')

In [ ]:
df_pop=df_pop.set_index('FIPS')

In [ ]:
def diff(dat, t,params):
    beta = params[0]
    delta = params[1]
    gamma = params[2]
    alpha = params[3]
    lambda_ = params[4]
    kappa = params[5]
    
    s = dat[0]
    e = dat[1]
    i = dat[2]
    q = dat[3]
    r = dat[4]
    d = dat[5]
    sa = dat[6]
    
    dsdt = - beta * s * i - alpha * s
    dedt = beta * s * i - gamma * e
    didt = gamma * e - lambda_ * i
    dqdt = lambda_ * i - delta * q - kappa * q
    drdt = delta * q
    dddt = kappa * q
    dsadt = alpha * s
    
    # susceptible, exposed, infected, quarantined, recovered, died, unsusceptible
    return [dsdt, dedt, didt, dqdt, drdt, dddt, dsadt]

In [ ]:
from scipy.integrate import odeint
def model(params, tmax, initial_death=0):
    # initial conditions
    initial_conditions = np.array(params[-5:]) # the parameters are a fraction of the population so multiply by the population
    
    # initial conditions
    e0 = initial_conditions[0]
    i0 = initial_conditions[1]
    q0 = initial_conditions[2]
    r0 = initial_conditions[3]
    sa0 = initial_conditions[4]
    
    d0 = initial_death
    s0 = 1 - np.sum(initial_conditions) - d0

    yz_0 = np.array([s0, e0, i0, q0, r0, d0, sa0])
    
    # Package parameters into a tuple
    args = (params,)
    
    
    # Integrate ODEs
    s = odeint(diff, yz_0, np.arange(0, tmax), args=args)

    return s

In [ ]:
def death_cost(params, Ddata, Idata):
    s = model(params, tmax=len(Ddata))
    D = s[:,5]
    I = s[:,2]
    error = np.concatenate((D-Ddata, I - Idata))
    return error

In [ ]:
from scipy.optimize import least_squares
import numpy as np
df = pd.read_csv(datadir + 'us/covid/confirmed_cases.csv')

In [ ]:
df2 = pd.read_csv(datadir + 'us/covid/deaths.csv')

In [ ]:
df.columns

In [ ]:
ind = np.logical_and(df['5/2/20']>100, df2['5/2/20']>10)

In [ ]:
x=df[df['4/23/20']>1000].sample()

In [ ]:
df[ind]

In [ ]:
xs=np.array(x.iloc[:,4:].values.flatten(),dtype=float)

In [ ]:
guesses= [1, 0.3, 0.2, 0.2, 0.2, 0.03, 0.5e-3, 0.5e-3, 0.3e-3, 0.1e-4, 0.5]

In [ ]:
from bokeh.models import Slider
slider = Slider(start=1, end=20, value=1, step=1, title="Minus days")

In [ ]:
slider.js_link('value', r.glyph, 'radius')

In [ ]:
from bokeh.plotting import figure, output_file, show
p = figure(title="Death tolls", plot_width=300, plot_height=300)
res = least_squares(death_cost, guesses, args=(xs[:-ns],))
p.line(x=model(res.x,tmax=len(xs))[:,5])
p.line(xs)
show(p)

In [ ]:
param_ranges = [(0, np.inf)]*6
initial_ranges = [(0,1)]*5

In [ ]:
tmp=df.iloc[15,4:]
tmp[tmp>0].index[0]

In [ ]:
def get_first_nonzero(s):
    tmp = np.nonzero(s.values)[1]
    if len(tmp) > 0:
        return s.columns[tmp[0]]
    else:
        return None

In [ ]:
first = []
for i in X.index:
    first.append(get_first_nonzero(i))

In [ ]:
nth_to_date = {i:df.columns[4+i] for i in range(93)}

In [ ]:
get_first_nonzero(df[df['countyFIPS']==51153].iloc[:,4:])

In [ ]:
x_i = df[df['countyFIPS']==4019]
x_d = df2[df2['countyFIPS']==4019]

In [ ]:
x_ds = x_d.loc[:,padding_i:].values.flatten()
x_is = x_i.loc[:,padding_i:].values.flatten()

In [ ]:
df.loc[ind,'countyFIPS']

In [ ]:
df_pop = df_pop.set_index('FIPS')

In [ ]:
df_pop

In [ ]:
def predict_selected_county(countyFIPS):
    N = df_pop.loc[countyFIPS, 'total_pop']
    x_i = df[df['countyFIPS']==countyFIPS]
    x_d = df2[df2['countyFIPS']==countyFIPS]
    padding_i = 'get_first_nonzero(x_i.iloc[:,4:])'
    print("First case: {}".format(padding_i))
    padding_d = get_first_nonzero(x_d.iloc[:,4:])
    print("First death: {}".format(padding_d))
    x_ds = x_d.loc[:,padding_i:].values.flatten()/N
    x_is = x_i.loc[:,padding_i:].values.flatten()/N
    # print("Deaths from date of first death: {}".format(xs))
    plt.figure(figsize=(20,10))
    for ns in range(1,20,2):
        res = least_squares(death_cost, guesses,
                            args=(x_ds[:-ns],x_is[:-ns]),
                            bounds=np.transpose(np.array(param_ranges+initial_ranges))
                           )
        pred = model(res.x,tmax=len(x_is))
        plt.plot(pred[:,5]*N,alpha=1-(ns-1)/20,c='r')
        plt.plot(pred[:,2]*N,alpha=1-(ns-1)/20,c='b')
    plt.plot(x_ds*N,c='r')
    plt.plot(x_is*N,c='b')
    plt.title("Death predictions for {name}, {state}".format(name=x_i['County Name'].values[0],state=x_i['State'].values[0]))
    plt.show()

In [ ]:
def predict_selected_county_scales(countyFIPS):
    N = df_pop.loc[countyFIPS, 'total_pop']
    x_i = df[df['countyFIPS']==countyFIPS]
    x_d = df2[df2['countyFIPS']==countyFIPS]
    padding_i = get_first_nonzero(x_i.iloc[:,4:])
    print("First case: {}".format(padding_i))
    padding_d = get_first_nonzero(x_d.iloc[:,4:])
    print("First death: {}".format(padding_d))
    x_ds = x_d.loc[:,padding_i:].values.flatten()/N
    x_is = x_i.loc[:,padding_i:].values.flatten()/N
    # print("Deaths from date of first death: {}".format(xs))
    fig, ax1 = plt.subplots(figsize=(20,10))
    ax2 = ax1.twinx()
    for ns in range(1,20,2):
        res = least_squares(death_cost, guesses,
                            args=(x_ds[:-ns],x_is[:-ns]),
                            bounds=np.transpose(np.array(param_ranges+initial_ranges))
                           )
        pred = model(res.x,tmax=len(x_is))
        ax1.plot(pred[:,5]*N,alpha=1-(ns-1)/20,c='r')
        ax2.plot(pred[:,2]*N,alpha=1-(ns-1)/20,c='b')
    ax1.plot(x_ds*N,c='r')
    ax2.plot(x_is*N,c='b')
    ax1.set_ylabel('Total Deaths', c='r')
    ax2.set_ylabel('Total Cases', c='b')
    plt.title("Death/Cases predictions for {name}, {state}".format(name=x_i['County Name'].values[0],state=x_i['State'].values[0]))
    plt.show()

In [ ]:
def predict_selected_county_scales_no_offset(countyFIPS):
    N = df_pop.loc[countyFIPS, 'total_pop']
    x_i = df[df['countyFIPS']==countyFIPS]
    x_d = df2[df2['countyFIPS']==countyFIPS]
    padding_i = '1/22/20'
    padding_d = '1/22/20'
    x_ds = x_d.loc[:,padding_i:].values.flatten()/N
    x_is = x_i.loc[:,padding_i:].values.flatten()/N
    # print("Deaths from date of first death: {}".format(xs))
    fig, ax1 = plt.subplots(figsize=(20,10))
    ax2 = ax1.twinx()
    for ns in range(1,20,2):
        res = least_squares(death_cost, guesses,
                            args=(x_ds[:-ns],x_is[:-ns]),
                            bounds=np.transpose(np.array(param_ranges+initial_ranges))
                           )
        pred = model(res.x,tmax=100)
        ax1.plot(pred[:,5]*N,alpha=.5-(ns-1)/40,c='r')
        ax2.plot(pred[:,2]*N,alpha=.5-(ns-1)/40,c='b')
    ax1.plot(x_ds*N,c='r',lw=2)
    ax2.plot(x_is*N,c='b',lw=2)
    ax1.set_ylabel('Total Deaths', c='r')
    ax2.set_ylabel('Total Cases', c='b')
    plt.title("Death/Cases predictions for {name}, {state}".format(name=x_i['County Name'].values[0],state=x_i['State'].values[0]))
    plt.show()

In [ ]:
predict_selected_county_scales(42079)

In [ ]:
def predict_selected_county_before_window2(countyFIPS, n_days_before):
    N = df_pop.loc[countyFIPS, 'total_pop']
    x_i = df[df['countyFIPS']==countyFIPS]
    x_d = df2[df2['countyFIPS']==countyFIPS]
    padding_i = get_first_nonzero(x_i.iloc[:,4:])
    padding_d = get_first_nonzero(x_d.iloc[:,4:])
    x_ds = x_d.loc[:,padding_i:].values.flatten()/N
    x_is = x_i.loc[:,padding_i:].values.flatten()/N
    res = least_squares(death_cost, guesses,args=(x_ds[:-n_days_before],x_is[:-n_days_before]),bounds=np.transpose(np.array(param_ranges+initial_ranges)))
    return res

In [ ]:
X_correlations = pd.DataFrame(index=df.loc[ind,'countyFIPS'].values)
X_correlations.head()

In [ ]:
predict_selected_county_before_window2(6059,1).x

In [ ]:
counter = 0
for f in X_correlations.index:
    tmp = predict_selected_county_before_window2(f, 1)
    for i in range(11):
        X_correlations.loc[f, 'param'+str(i)]=tmp.x[i]
    # print(counter)
    counter += 1

In [ ]:
X_correlations.index

In [ ]:
X_correlations

In [ ]:
X_correlations=X_correlations.dropna()

### Check fitting

In [ ]:
predict_selected_county(4013)

In [ ]:
plt.plot(model(X_correlations.loc[4013].values,98)[:,2]*df_pop.loc[4013,'total_pop'])

In [ ]:
plt.plot(df[df['countyFIPS']==4013].loc[:,'1/26/20':].values.flatten())

## Correlation plots

### Beds

In [ ]:
df_beds = pd.read_csv(datadir + 'us/hospitals/beds_by_county.csv')

In [ ]:
X_bed_tot = pd.merge(X_correlations, df_beds, left_index=True, right_on='FIPS').drop(columns=['state','county','Name']).set_index('FIPS')

In [ ]:
import seaborn as sns

In [ ]:
cmap = sns.diverging_palette(10,
                             240,
                             #as_cmap=True,
                             n=11,
                             sep=1)

In [ ]:
sns.heatmap(X_bed_tot.corr(), vmin=-1, vmax=1,cmap=cmap, linewidths=1, square=True)
plt.title("Correlations between parameters and beds from counties with > 1000 deaths in the last day")
plt.savefig("corr_beds.png")

In [ ]:
#plt.figure(figsize=(30,5))
ax = sns.heatmap(X_bed_tot.corr().loc[:'param10', 'staffed_beds':], vmin=-1, vmax=1,cmap=cmap, linewidths=1, square=True)
locs, labels = plt.yticks()
plt.yticks(locs, [r"$\beta$",
                  r"$\delta$",
                  r"$\gamma$",
                  r"$\alpha$",
                  r"$\lambda$",
                  r"$\kappa$",
                  r"$E(t=0)$",r"$I(t=0)$",r"$Q(t=0)$",
                  r"$R(t=0)$",r"$S_A(t=0)$"]
          )
plt.title("Correlations between parameters and beds with > 1000 deaths in the last day")
plt.savefig("corr_beds.png")

### Mobility

In [ ]:
df_mob = pd.read_csv(datadir + 'us/mobility/DL-us-m50.csv')

In [ ]:
df_mob_ind = pd.read_csv(datadir + 'us/mobility/DL-us-m50_index.csv')
df_mob_ind.head()

In [ ]:
df_mob_ind['mean1']=np.mean(df_mob_ind[df_mob_ind.columns[5:15]],axis=1)
df_mob_ind['mean2']=np.mean(df_mob_ind[df_mob_ind.columns[15:25]],axis=1)
df_mob_ind['mean3']=np.mean(df_mob_ind[df_mob_ind.columns[25:35]],axis=1)
df_mob_ind['mean4']=np.mean(df_mob_ind[df_mob_ind.columns[35:45]],axis=1)

In [ ]:
df_mob_ind.head()

In [ ]:
df_mob_ind['mean5']=np.mean(df_mob_ind[df_mob_ind.columns[45:-5]],axis=1)

In [ ]:
df_mob['fips'].dropna().astype(int)

In [ ]:
df_mob_ind=df_mob_ind.dropna(subset=['fips'])

In [ ]:
df_mob_ind['FIPS'] = df_mob_ind['fips'].dropna().astype(int)

In [ ]:
df_mob_ind

In [ ]:
df_mob_ind2=pd.DataFrame(df_mob_ind[['FIPS', 'mean1', 'mean2', 'mean3', 'mean4', 'mean5']])

In [ ]:
X_mobind_tot=pd.merge(df_mob2, X_correlations,left_on='FIPS', right_index=True).set_index('FIPS')

In [ ]:
sns.heatmap(X_mobind_tot.corr(), vmin=-1, vmax=1,cmap=cmap, linewidths=1, square=True)
plt.title("Correlations between parameters and mean mobility from counties with > 1000 deaths in the last day")
plt.savefig("corr_mob.png")

In [ ]:
X_mobind_tot=pd.merge(df_mob_ind2, X_correlations,left_on='FIPS', right_index=True).set_index('FIPS')

In [ ]:
sns.heatmap(X_mobind_tot.corr(), vmin=-1, vmax=1,cmap=cmap, linewidths=1, square=True)

plt.title("Correlations between parameters and mean mobility index from counties with > 1000 deaths in the last day")
plt.savefig("corr_mob.png")

In [ ]:
sns.heatmap(X_mobind_tot.corr().loc['param0':, :'mean5'], vmin=-1, vmax=1,cmap=cmap, linewidths=1, square=True)
locs, labels = plt.yticks()
plt.yticks(locs, [r"$\beta$",
                  r"$\delta$",
                  r"$\gamma$",
                  r"$\alpha$",
                  r"$\lambda$",
                  r"$\kappa$",
                  r"$E(t=0)$",r"$I(t=0)$",r"$Q(t=0)$",
                  r"$R(t=0)$",r"$S_A(t=0)$"]
          )
plt.title("Correlations between parameters and mean mobility index from counties with > 1000 deaths in the last day")
plt.savefig("corr_mob.png")

In [ ]:
df_mob_ind.columns[-7]

In [ ]:
plt.figure(figsize=(20,10))
sns.distplot(np.clip(df_mob_ind2['mean1'],0,200),bins=20)
sns.distplot(np.clip(df_mob_ind2['mean2'],0,200),bins=20)
sns.distplot(np.clip(df_mob_ind2['mean3'],0,200),bins=20)
sns.distplot(np.clip(df_mob_ind2['mean4'],0,200),bins=20)
sns.distplot(np.clip(df_mob_ind2['mean5'],0,200),bins=20)
plt.legend(('2020-03-01'+'~'+'2020-03-10','2020-03-11'+'~'+'2020-03-20','2020-03-21'+'~'+'2020-03-30','2020-03-31'+'~'+'2020-04-10','2020-04-11'+'~'+'2020-04-21'))
plt.title('Changes in mobility')
plt.savefig('mobility_changes.png')

### Berkeley Aggregate

In [ ]:
df_berkeley = pd.read_csv(datadir + 'us/aggregate_berkeley.csv').set_index('Unnamed: 0').reset_index(drop=True)
df_berkeley.head()

In [ ]:
df_berkeley.columns

In [ ]:
for keys in ['PopMale<52010',
       'PopFmle<52010', 'PopMale5-92010', 'PopFmle5-92010', 'PopMale10-142010',
       'PopFmle10-142010', 'PopMale15-192010', 'PopFmle15-192010',
       'PopMale20-242010', 'PopFmle20-242010', 'PopMale25-292010',
       'PopFmle25-292010', 'PopMale30-342010', 'PopFmle30-342010',
       'PopMale35-442010', 'PopFmle35-442010', 'PopMale45-542010',
       'PopFmle45-542010', 'PopMale55-592010', 'PopFmle55-592010',
       'PopMale60-642010', 'PopFmle60-642010', 'PopMale65-742010',
       'PopFmle65-742010', 'PopMale75-842010', 'PopFmle75-842010',
       'PopMale>842010', 'PopFmle>842010']:
    df_berkeley[keys] = df_berkeley[keys]/df_berkeley['PopulationEstimate2018']

In [ ]:
for keys in ['3-YrMortalityAge<1Year2015-17',
       '3-YrMortalityAge1-4Years2015-17', '3-YrMortalityAge5-14Years2015-17',
       '3-YrMortalityAge15-24Years2015-17',
       '3-YrMortalityAge25-34Years2015-17',
       '3-YrMortalityAge35-44Years2015-17',
       '3-YrMortalityAge45-54Years2015-17',
       '3-YrMortalityAge55-64Years2015-17',
       '3-YrMortalityAge65-74Years2015-17',
       '3-YrMortalityAge75-84Years2015-17', '3-YrMortalityAge85+Years2015-17',
       'mortality2015-17Estimated']:
    df_berkeley[keys] = df_berkeley[keys]/df_berkeley['Population(Persons)2017']

In [ ]:
X_berkeley_tot = pd.merge(X_correlations, df_berkeley, left_index=True, right_on='countyFIPS').drop(columns=['State', 'County']).set_index('countyFIPS')

0. $\beta\quad(S\times I\to E)$
1. $\delta\quad(Q\to R)$
2. $\gamma\quad(E\to I)$
3. $\alpha\quad(S\to S_A)$
4. $\lambda\quad(I\to Q)$
5. $\kappa\quad(Q\to D)$
6. $E(t=0)$
7. $I(t=0)$
8. $Q(t=0)$
9. $R(t=0)$
10. $S_A(t=0)$


In [ ]:
plt.figure(figsize=(30,5))
ax = sns.heatmap(X_berkeley_tot.corr().loc[:'param10', 'PopulationEstimate2018':], vmin=-1, vmax=1,cmap=cmap, linewidths=1, square=True)
locs, labels = plt.yticks()
plt.yticks(locs, [r"$\beta$",
                  r"$\delta$",
                  r"$\gamma$",
                  r"$\alpha$",
                  r"$\lambda$",
                  r"$\kappa$",
                  r"$E(t=0)$",r"$I(t=0)$",r"$Q(t=0)$",
                  r"$R(t=0)$",r"$S_A(t=0)$"]
          )
plt.title("Correlations between parameters and berkeley aggregates with > 1000 deaths in the last day")
plt.savefig("corr_berkeley_zoomin_discrete.png")

In [ ]:
plt.figure(figsize=(25,20))
sns.heatmap(X_berkeley_tot.corr(), vmin=-1, vmax=1,cmap=cmap, linewidths=1, square=True)
plt.title("Correlations between parameters and berkeley aggregates with > 1000 deaths in the last day")
plt.savefig("corr_berkeley.png")

# Regression on SEIR parameters

## Model exploration

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, BayesianRidge

In [ ]:
X_bed_tot

In [ ]:
X_tot

In [ ]:
rf_Y = X_bed_tot.loc[:,'param4'].reset_index(drop=True)
rf_X = X_bed_tot.loc[:,'staffed_beds':].reset_index(drop=True)

In [ ]:
# rf_Y = df.loc[rf_X.index,'5/2/20']

In [ ]:
from sklearn.model_selection import KFold
def cv_estimate(n_splits=None):
    cv = KFold(n_splits=n_splits)
    cv_clf = LinearRegression()
    for train, test in cv.split(rf_X, rf_Y):
        cv_clf.fit(rf_X.iloc[train], rf_Y.iloc[train])
        print(cv_clf.score(rf_X.iloc[test], rf_Y.iloc[test]))
    return None

In [ ]:
cv_estimate(3)

## Hyperparameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
clf = GridSearchCV(rfc, {'max_depth': [10, 20], 'min_impurity_decrease': [0.05, 0.1]})
clf.fit(rf_X,rf_Y)

In [ ]:
clf.cv_results_

## Permutation importance

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(rf_X, rf_Y, test_size=0.33, random_state=42)

In [ ]:
reg = LinearRegression()
reg.fit(X_train,y_train)
result = permutation_importance(reg, X_test, y_test, random_state=10, n_repeats=15, n_jobs=8)

In [ ]:
sorted_idx = result.importances_mean.argsort()[:10]
fig, ax = plt.subplots()
fig.set_size_inches(11,8)
ax.boxplot(result.importances[sorted_idx].T,
           vert=False, labels=X_test.columns[sorted_idx])
ax.set_title("Permutation Importances (test set)")
fig.tight_layout()
plt.show()

## Partial dependency plot

In [ ]:
from sklearn.inspection import plot_partial_dependence

In [ ]:
X_bed_tot.columns

In [ ]:
plot_partial_dependence(reg, X_train, [sorted_idx[0]])
plot_partial_dependence(reg, X_train, [sorted_idx[1]])
plot_partial_dependence(reg, X_train, [sorted_idx[2]])

In [ ]:
fig = plt.figure(figsize=(20,10))
plot_partial_dependence(reg, X_train,[(sorted_idx[0],sorted_idx[1])])
plt.show()

## SHAP